# Tugas 1a: Melakukan Pembuatan Teks

Dalam <i>notebook</i> ini, Anda mempelajari cara menggunakan Model Bahasa Besar (LLM) untuk menghasilkan respons email kepada pelanggan yang memberikan umpan balik negatif tentang kualitas layanan pelanggan yang diterima dari teknisi dukungan. Di <i>notebook </i>ini, Anda membuat email dengan catatan terima kasih berdasarkan email pelanggan sebelumnya. Anda memakai model Amazon Titan yang menggunakan API Amazon Bedrock dengan klien Boto3.

Perintah yang digunakan dalam tugas ini disebut perintah <i>zero-shot</i>. Dalam perintah <i>zero-shot</i>, Anda menjelaskan tugas atau <i>output</i> yang diinginkan ke model bahasa dalam bahasa sederhana. Model tersebut kemudian menggunakan pengetahuan dan kemampuan yang telah dilatih sebelumnya untuk menghasilkan respons atau menyelesaikan tugas hanya berdasarkan perintah yang disediakan.

#### Skenario
Anda adalah Bob, Manajer Layanan Pelanggan di AnyCompany. Beberapa pelanggan Anda tidak puas dengan layanan pelanggan dan memberikan umpan balik negatif terhadap layanan yang diberikan oleh teknisi dukungan pelanggan. Sekarang, Anda ingin merespons umpan balik pelanggan untuk meminta maaf atas layanan yang buruk dan mendapatkan kembali kepercayaan. Anda memerlukan bantuan LLM untuk membuat sejumlah besar email yang ramah manusia dan disesuaikan dengan sentimen pelanggan dari korespondensi email sebelumnya.

## Tugas 1a.1: Pengaturan lingkungan

Dalam tugas ini, Anda mengatur lingkungan Anda.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Tugas 1a.2: Membuat teks

Dalam tugas ini, Anda menyiapkan input untuk layanan Amazon Bedrock untuk membuat email.

In [ ]:
# create the prompt
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, AnyCompany to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""

In [ ]:
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":8192,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

Selanjutnya, Anda menggunakan model Amazon Titan.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Catatan:** Amazon Titan mendukung jendela konteks ~4k token dan menerima parameter berikut:
- `inputText`: Perintah untuk LLM
- `textGenerationConfig`: Ini adalah parameter yang akan dipertimbangkan model saat menghasilkan <i>output</i>.

API Amazon Bedrock memberi Anda API `invoke_model` yang menerima:
- `modelId`: Ini adalah model ARN untuk berbagai model fondasi yang tersedia di Amazon Bedrock
- `accept`: Jenis permintaan input
- `contentType`: Jenis konten <i>output</i>
- `body`: String json berisi perintah dan konfigurasi

Lihat [dokumentasi] (https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) untuk ID model pembuatan teks yang tersedia.

## Tugas 1a.3: Menginvokasi model bahasa Amazon Titan Large

Dalam tugas ini, Anda akan mempelajari cara model menghasilkan <i>output</i> berdasarkan perintah yang dibuat sebelumnya.

### Menyelesaikan Pembuatan <i>Output</i>

Email ini dibuat menggunakan model Amazon Titan dengan memahami permintaan input dan memanfaatkan pemahaman yang melekat pada modalitas yang berbeda. Permintaan ke API ini sinkron dan menunggu seluruh <i>output</i> dihasilkan oleh model.

In [ ]:
#invoke model
modelId = 'amazon.titan-text-express-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"
try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [ ]:
# The relevant portion of the response begins after the first newline character
# Below we print the response beginning after the first occurence of '\n'.

email = outputText[outputText.index('\n')+1:]
print(email)


### <i>Streaming</i> Pembuatan <i>Output</i>

Bedrock juga mendukung <i>streaming output</i> karena dihasilkan oleh model dalam bentuk potongan. Email ini dibuat dengan memanggil model dengan opsi streaming. `invoke_model_with_response_stream` menampilkan `ResponseStream` yang dapat Anda baca.

In [ ]:
# invoke model with response stream
output = []
try:
    
    response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                output.append(text)
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

<i>Stream</i> dengan pendekatan respons membantu mendapatkan <i>output</i> model dengan cepat dan memungkinkan layanan untuk menyelesaikannya saat Anda membaca. Hal ini membantu dalam kasus penggunaan ketika Anda meminta model menghasilkan potongan teks yang lebih panjang. Nantinya, Anda dapat menggabungkan semua potongan yang dihasilkan untuk membentuk <i>output</i> lengkap dan menggunakannya untuk kasus penggunaan Anda. 

In [ ]:
#combine output chunks
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)


Kini Anda telah bereksperimen menggunakan SDK boto3 yang memberikan paparan dasar mengenai API Amazon Bedrock. Dengan menggunakan API ini, Anda telah melihat kasus penggunaan pembuatan email untuk merespons umpan balik negatif pelanggan.

### Cobalah sendiri
- Ubah perintah untuk kasus penggunaan spesifik Anda dan evaluasi <i>output</i> model yang berbeda.
- Atur panjang token untuk memahami latensi dan responsivitas layanan.
- Terapkan berbagai prinsip rekayasa perintah untuk mendapatkan <i>output</i> yang lebih baik.

### Pembersihan

Anda telah menyelesaikan <i>notebook</i> ini. Untuk pindah ke bagian lab berikutnya, lakukan hal berikut:

- Tutup <i>file notebook</i> ini dan lanjutkan dengan **Task1b.ipynb**.